# Notebook for model training

### Training Flow

Load Training Data:
- Load set of input data and label files
- Prefilter positive set only to rule out heavily-masked patches
- Create train/test set
- Define augmentation parameters

Train Model:
- Define model architecture
- Compile model
- Train and evaluate model
- Save model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import date
from datetime import datetime
import time
import os
import pickle
import sys

import pandas as pd
import io
from contextlib import redirect_stdout
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model

from keras.layers import Input, Dense, Flatten, Dropout, Conv2D
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tqdm.notebook import tqdm

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import dl_utils
from scripts import viz_tools
from scripts import models

import gc
from tensorflow.keras import backend as K

# export TF_GPU_ALLOCATOR=cuda_malloc_async
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

2024-09-25 17:19:10.868451: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 17:19:10.890912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 17:19:10.890938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 17:19:10.891667: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 17:19:10.896077: I tensorflow/core/platform/cpu_feature_guar

## Open Data

In [3]:
resolution = 48

data_files = [
    "amazonas_2020_thresh_0.5_2_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "amazonas_2020_thresh_0.8_sumbsample_3_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "MinesPos2018-2020Sentinel_points_2019-01-01_2020-01-01_patch_arrays.pkl",
    "bolivar_2020_thresh_0.8_sumbsample_5_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "full_amazon_v9_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.4_amazon_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.6_amazon_thresh_0.8_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.0_bolivar_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "riverbank_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "bolivar_2020_thresh_0.8_1_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.1.1_bolivar_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.1_bolivar_positives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.4_amazonas_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.4_amazon_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.6_amazon_negatives_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v2.6_amazon_negatives_v2_2019-01-01_2020-01-01_patch_arrays.pkl",
    "v3.1_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "amazon_all_48px_v3.1_2023_positives_0.999_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.2_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.3_2023_positives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.4_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.5_2023_negatives_2023-01-01_2024-01-01_patch_arrays.pkl",
    "v3.6_2023_positives_2023-01-01_2024-01-01_patch_arrays.pkl"
    
    ]

label_files = [
    "amazonas_2020_thresh_0.5_2_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "amazonas_2020_thresh_0.8_sumbsample_3_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "MinesPos2018-2020Sentinel_points_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "bolivar_2020_thresh_0.8_sumbsample_5_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "full_amazon_v9_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.4_amazon_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.6_amazon_thresh_0.8_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.0_bolivar_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "riverbank_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "bolivar_2020_thresh_0.8_1_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.1.1_bolivar_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.1_bolivar_positives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.4_amazonas_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.4_amazon_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.6_amazon_negatives_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v2.6_amazon_negatives_v2_2019-01-01_2020-01-01_patch_array_labels.pkl",
    "v3.1_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "amazon_all_48px_v3.1_2023_positives_0.999_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.2_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.3_2023_positives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.4_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.5_2023_negatives_2023-01-01_2024-01-01_patch_array_labels.pkl",
    "v3.6_2023_positives_2023-01-01_2024-01-01_patch_array_labels.pkl"
    
    
]

patches = []
labels = []

data_dir = os.path.join('..', 'data', 'training_data', f"{resolution}_px")

for data, label in tqdm(zip(data_files, label_files), total=len(data_files)):
    with open(os.path.join(data_dir, data), 'rb') as f:
        data = pickle.load(f)
        for elem in data:
            #patch = dl_utils.pad_patch(elem, resolution)
            patches.append(elem)
    with open(os.path.join(data_dir, label), 'rb') as f:
        label = pickle.load(f)
        labels = np.concatenate((labels, label))

patches = np.array(patches)
negative_patches = patches[labels == 0]
positive_patches = patches[labels == 1]

print(len(patches), "samples loaded")
print(sum(labels == 1), "positive samples")
print(sum(labels == 0), "negative samples")

  0%|          | 0/23 [00:00<?, ?it/s]

4549 samples loaded
1891 positive samples
2658 negative samples


In [4]:
num_samples = 64
indices = np.random.randint(0, len(patches), num_samples)
# viz_tools.plot_image_grid(patches[indices], labels=[int(label) for label in labels[indices]])

In [5]:
def filter_black(data, mask_limit=0.1, return_rejects=False):
    masked_fraction = np.array([np.sum(np.mean(patch, axis=-1) < 10) / np.size(np.mean(patch, axis=-1)) for patch in data])
    filtered_data = data[masked_fraction < mask_limit]
    print(f"{len(filtered_data) / len(data) :.1%} of data below brightness limit")
    if return_rejects:
        rejected_data = data[masked_fraction >= mask_limit]
        return filtered_data, rejected_data
    else:
        return filtered_data

In [6]:
# Filter positive pixels that are masked beyond a threshold. Don't want to give positive examples of cloud-masked patches
filtered_positives, positive_rejects = filter_black(positive_patches, mask_limit = 0.1, return_rejects=True)
if len(positive_rejects) > 0:
    num_samples = 16**2
    if len(positive_rejects) < num_samples:
        num_samples = len(positive_rejects)
    indices = np.random.randint(0, len(positive_rejects), num_samples)
    # viz_tools.plot_numpy_grid(positive_rejects[indices,:,:,3:0:-1] / 3000)
    # plt.title("Positive Masked Rejects")
    # plt.show()

num_samples = 25 ** 2
indices = np.random.randint(0, len(filtered_positives), num_samples)
# fig = viz_tools.plot_numpy_grid(filtered_positives[indices,:,:,3:0:-1] / 3000)
# plt.title('Positive Filtered Samples')
# plt.show()

99.9% of data below brightness limit


In [7]:
filtered_negatives, negative_rejects = filter_black(negative_patches, mask_limit = 0.6, return_rejects=True)
if len(negative_rejects) > 0:
    num_samples = 16**2
    if len(positive_rejects) < num_samples:
        num_samples = len(positive_rejects)
    indices = np.random.randint(0, len(negative_rejects), num_samples)
    # viz_tools.plot_numpy_grid(negative_rejects[indices,:,:,3:0:-1] / 3000)
    # plt.title("Negative Mask Rejects")
    # plt.show()

num_samples = 25 ** 2
indices = np.random.randint(0, len(filtered_negatives), num_samples)
# fig = viz_tools.plot_numpy_grid(filtered_negatives[indices,:,:,3:0:-1] / 3000)
# plt.title('Negative Filtered Samples')
# plt.show()

100.0% of data below brightness limit


## Prepare Data for Training

In [8]:
#for RGBIR, x = normalize(np.copy(images[:,:,:,[1,2,3,8]]))
x = np.concatenate((filtered_negatives, filtered_positives))
y = np.concatenate((np.zeros(len(filtered_negatives)), np.ones(len(filtered_positives))))
x, y = shuffle(x, y, random_state=33)

In [9]:
augmentation_parameters = {
    'featurewise_center': False,
    'rotation_range': 360,
    'width_shift_range': [0.9, 1.1],
    'height_shift_range': [0.9, 1.1],
    'shear_range': 10,
    'zoom_range': [0.9, 1.1],
    'vertical_flip': True,
    'horizontal_flip': True,
    # Fill options: "constant", "nearest", "reflect" or "wrap"
    'fill_mode': 'reflect'
}

datagen = ImageDataGenerator(**augmentation_parameters)


plt.figure(figsize=(12,12), facecolor=(1,1,1), dpi=150)
aug_img, aug_labels = datagen.flow(x, y, batch_size=64).next()
# viz_tools.plot_image_grid(aug_img, labels=[int(l) for l in aug_labels], norm=True);


/home/raulpg/anaconda3/envs/tfm_tf/lib/python3.11/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4548, 48, 48, 12) (12 channels).
  warnings.warn(


<Figure size 1800x1800 with 0 Axes>

In [10]:
x_norm = np.clip(np.array(x.astype("float32") / 10000), 0, 1)

Partimos el 10% del dataset total para test

In [11]:
dataset_size = len(y)
test_partition = int(dataset_size*0.1)

In [12]:
num_positive_samples_train = sum(y[test_partition:]==1)
num_negative_samples_train = sum(y[test_partition:]==0)

print(f"Número de clases positivas en el dataset de training es de {num_positive_samples_train}, número de clases negativas {num_negative_samples_train}")

Número de clases positivas en el dataset de training es de 1705, número de clases negativas 2389


Eliminamos estos samples del dataset de validacion

In [13]:
test_partition_x = x_norm[:test_partition]
test_partition_y = y[:test_partition]

x_norm = x_norm[test_partition:]
y = y[test_partition:]

# Entrenamiento de la arquitectura original con los parámetros originales

In [14]:
hiper_params_original_test = {"class_weight" : {0: 1, 1: 1},
                              "batch_size" : 32,
                              "epochs" : 160,
                              "test_size" : 0.3
                           }
input_shape = (48,48,12)

### Entrenamiento
- Entrenamos el modelo original para 5 semillas distintas. Se elige el mejor modelo para poder compararlo con los demás.
- Total de modelos construidos = 5

In [17]:
%%time

# Descripcion de esta prueba
desc_test = "modelo_y_parametros_originales"
hiperparametros = [hiper_params_original_test]

# Inicializamos para dar nombre a todos los outputs que se escriban
hora_exec = datetime.now().strftime('%Y-%m-%d_%H-%M')

# Creamos el nombre del directorio para guardar los modelos finales
path_prueba_actual = f"../notebooks/best_models/{desc_test}_{hora_exec}/"
os.makedirs(path_prueba_actual, exist_ok=True)

# Inicializamos las variables dónde almacenaremos los resultados de cada uno de los modelos generados
df_scratch = pd.DataFrame()

#Eliminamos los csvs generados en ejecuciones anteriores
dl_utils.limpia_directorios("aux_path")

# Recorremos la lista de configuraciones distinta generada anteriormente
for id_hiperparams,hiperparams in enumerate(hiperparametros):
    for semilla in range(0,1):  

        # Limpiamos la carpeta donde se guardan los modelos
        dl_utils.limpia_directorios("weigths")

        # Definimos hiperparámetros asociados a la ejecución actual
        batch_size = hiperparams["batch_size"]
        epochs = hiperparams["epochs"]
     
        # Dividimos el conjunto de imágenes en train y test
        seed = np.random.randint(0, 2000,1)[0]
        hiperparams["semilla"] = seed
        keras.utils.set_random_seed(int(seed))
        x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=hiperparams["test_size"], random_state=123)

        # Definimos y compilamos los modelos
        scratch_original_model_and_params = models.scratch_original_model(input_shape)

        start_time = time.time()
        # Modelo por defecto empleando los mismos parámetros que el original.
        scratch_original_model_and_params_results = scratch_original_model_and_params.fit(
            datagen.flow(x_train, y_train),
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            verbose=1,
            shuffle=True,
            class_weight=hiperparams["class_weight"],
        )
        training_time_scratch_original_model_and_params = time.time() - start_time

        # Evaluamos el modelo y lo comparamos con el mejor almacenado.  
        hora_actual = datetime.now().strftime('%Y-%m-%d_%H-%M')
        df_scratch,          save_df_scratch          = dl_utils.get_df_best_model(scratch_original_model_and_params,x_test,y_test,f"scratch_original_model_and_params_{desc_test}_",df_scratch,hora_actual)

        # Si el modelo actual es mejor que el almacenado, guardamos su información.      
        if save_df_scratch:
            best_scratch_original_model_and_params = scratch_original_model_and_params
            best_scratch_original_model_and_params_results = scratch_original_model_and_params_results.history
            best_scratch_original_model_and_params_hiperparams = [hiperparams,
                                                                  id_hiperparams+1,
                                                                  f'resultados_evaluacion_scratch_original_model_and_params_{desc_test}_{hora_actual}.csv',
                                                                  training_time_scratch_original_model_and_params,
                                                                  len(scratch_original_model_and_params_results.epoch)]
        
        del scratch_original_model_and_params
        del scratch_original_model_and_params_results
        K.clear_session()
        gc.collect()        

# Guardamos el mejor modelo
dl_utils.save_models(best_scratch_original_model_and_params,best_scratch_original_model_and_params_results,best_scratch_original_model_and_params_hiperparams,hora_exec,f'scratch_original_model_and_params_{desc_test}',path_prueba_actual)

# Movemos los archivos asociados a los resultados
dl_utils.obtener_tablas_resultado(path_prueba_actual,[best_scratch_original_model_and_params_hiperparams[2]])

Epoch 1/160


/home/raulpg/anaconda3/envs/tfm_tf/lib/python3.11/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (2865, 48, 48, 12) (12 channels).
  warnings.warn(


90/90 [==============================] - 4s 25ms/step - loss: 0.6845 - acc: 0.5798 - val_loss: 0.6687 - val_acc: 0.5899
Epoch 2/160
90/90 [==============================] - 3s 30ms/step - loss: 0.6674 - acc: 0.6321 - val_loss: 0.6457 - val_acc: 0.7567
Epoch 3/160
90/90 [==============================] - 3s 32ms/step - loss: 0.6007 - acc: 0.7127 - val_loss: 0.5650 - val_acc: 0.6998
Epoch 4/160
90/90 [==============================] - 3s 29ms/step - loss: 0.5302 - acc: 0.7588 - val_loss: 0.5156 - val_acc: 0.7559
Epoch 5/160
90/90 [==============================] - 3s 29ms/step - loss: 0.5114 - acc: 0.7756 - val_loss: 0.4683 - val_acc: 0.7640
Epoch 6/160
90/90 [==============================] - 3s 29ms/step - loss: 0.5122 - acc: 0.7731 - val_loss: 0.5362 - val_acc: 0.7006
Epoch 7/160
90/90 [==============================] - 3s 29ms/step - loss: 0.4956 - acc: 0.7672 - val_loss: 0.4515 - val_acc: 0.7852
Epoch 8/160
90/90 [==============================] - 3s 30ms/step - loss: 0.4818 - acc: 